In [1]:
# !pip install pygame

In [2]:
import math
import pygame  ## useful libraries foe simulations

pygame 2.0.1 (SDL 2.0.14, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


### ENVIRONMENT

In [3]:
class buildEnvironment:  ## class for building the environment
    def __init__(self, MapDimensions):  ## init menthod specifying MapDimensions as arguments 
        pygame.init()  ## initialize pygame instance
        self.pointCloud =[]  ## list to store point cloud map (group od 2d points in 2d space)
        self.externalMap = pygame.image.load('map1.png') ## uploading our image file and storing in self.externalMap variable
        self.maph , self.mapw = MapDimensions  # splitting the dimensions of map into width and height
        self.MapWindowName = 'RRT path planning' ## naming window
        pygame.display.set_caption(self.MapWindowName)  ## setting window name
        self.map = pygame.display.set_mode((self.mapw , self.maph))  ## drawing self map which is empty white map of MapDimensions
        display (self.map)
        ## drawing external map on the top of it
        self.map.blit(self.externalMap,(0,0)) ## overlaying map1 using blit function
        #colors
        self.black = (0,0,0)
        self.grey = (70,70,70)
        self.Blue = (0,0,255)
        self.Green = (0,255,0)
        self.Red = (255,0,0)
        self.white = (255,255,255)
    
    def AD2pos(self,distance,angle,robotPosition): # to convert the sensor raw angle and distance data into cartesian coordinates
        x = distance*math.cos(angle) + robotPosition[0]
        y = -distance*math.sin(angle) + robotPosition[1]
        return(int(x),int(y)) ## return the output to be used by method dataStorage
    
    def dataStorage(self,data):  ## this method will take the raw data and use the AD2pos to convert it into cartesian coordinates
        print(len(self.pointCloud)) ## and store that point into point cloud list
        for element in data:
            point = self.AD2pos(element[0], element[1], element[2])
            if point not in self.pointCloud: ## after checking the duplicates it will append the data into cloud
                self.pointCloud.append(point)
    
    
    ## this class will draw our data onto our new map which will be first initialized to the main map
    def show_sensorData(self):
        self.infomap = self.map.copy()
        for point in self.pointCloud: ## in this for loop every point in the list will be drawn by setting the corresponding pixel to color red
            self.infomap.set_at((int(point[0]), int(point[1])), (255,0,0))

### MAIN

In [4]:
## environment = env.buildEnvironment((600,1200))
## If you run below lines  you will get the plot of map
# environment = buildEnvironment((600,1200))
# running = True

# while running:
#     for event in pygame.event.get():
#         if event.type == pygame.QUIT:
#             running = False
    
#     pygame.display.update()

### SENSORS

In [5]:
import numpy as np

def uncertainty_add(distance, angle, sigma): ## to measure noise, we take a random value in vicinity of actual measurement
    # example, if sensor o/p = 5 meter , sample a random value from a gaussian distribution with mean = 5
    # and variance is the uncertainity value we provide
    mean= np.array([distance,angle]) ## arranging output in numpy array
    covariance = np.diag(sigma**2)  ## using sigma to make covariance matrix
    distance, angle = np.random.multivariate_normal(mean, covariance) ## passing mean and covariance to get noisy values
    distance  = max(distance,0) ## to ensure we dont get any negative value
    angle = max(angle,0)
    return [distance, angle] ## returning our measurements as an array
    
    

class LaserSensor:
    
    def __init__(self,Range,Map,uncertainty): ## in init method we define 3 methods
        ## three arguments required for sensor measurements Range,map, uncertainty (uncertainity to adjust some error in measurement if any)
        self.Range = Range
        self.speed = 4  ## it can be used if we want to fix the speed of lidar rotation
        self.sigma = np.array([uncertainty[0],uncertainty[1]])  ## sigma is sensor measurement noise and since we have two componenets distance and angle, we will create two numpy array of uncertainity
        self.position = (0,0)  ## setting initial position(0,0) --- > i.e. top right corner of screen
        self.map = Map 
        self.W, self.H = pygame.display.get_surface().get_size()  ## getting window dimension
        self.sensedObstacles = []  ## list to store our point cloud
    
    
    def distance(self, obstaclePosition): ## function to find distance 
        px = (obstaclePosition[0] - self.position[0])**2
        py = (obstaclePosition[1] - self.position[1])**2
        return math.sqrt(px + py)
    
    
    def sense_obstacle(self):  ## sensing an obstacle, simple way is by sampling
        data = [] ## making list to store data
        x1, y1 = self.position[0], self.position[1] ## extracting x and y coordinates of robot
        for angle in np.linspace(0, 2*math.pi, 60, False):  ## for loop that iterates through a list made of angles between zero to 2 pi
            x2, y2 = (x1 + self.Range*math.cos(angle) , y1 - self.Range*math.sin(angle)) ## calculating the coordinates of the points that represents the end of the line segment
            for i in range(0,100):  ##sampling for 100 iteration
                u = i/100
                x = int(x2*u + x1*(1-u)) ## calculating the coordinates of line segment using interpolation formula
                y = int(y2*u + y1*(1-u))
                if 0<x<self.W and 0<y<self.H:  ## if point is within the window we will extract the color of the extact point from the map using 
                    color = self.map.get_at((x,y))
                    if (color[0],color[1],color[2]) == (0,0,0):  ## if the color is black(i.e. robot found an obstacle), we then add the distance of that point from robot
                        distance = self.distance((x,y))
                        output = uncertainty_add(distance, angle, self.sigma) ## and add the uncertainity to the returned array called output
                        output.append(self.position) ## append the position of the robot
                        data.append(output)  ## append the output list to data
                        break ## break because we encountered with an obstacle
        
#         when sensor took full circle we return the data in the map
        if len(data)>0:
            return data
        else:
            return False

### MAIN

In [6]:
environment = buildEnvironment((600,1200))  ## build the environent
environment.originalMap = environment.map.copy() ## save the copy of the main map that has plan as original map
laser = LaserSensor(200,environment.originalMap, uncertainty = (0.5,0.01)) ## create laser sensor instance providing the range and uncertainity values
environment.map.fill((0,0,0)) ##and then we will fill the map by black color
environment.infomap = environment.map.copy() # name it as info map and on this we will draw our point cloud in

running = True

while running:
    sensorON =False ## sensor is on only if mouse cursor is inside the window
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if pygame.mouse.get_focused():
            sensorON = True
        elif not pygame.mouse.get_focused():
            sensorON = False
    
    if sensorON:
        position = pygame.mouse.get_pos() ## getting mouse position
        laser.position = position ## assiging this position as sensor position
        sensor_data = laser.sense_obstacle() ## we run the lidar sensor for 1 turn
        environment.dataStorage(sensor_data) ## store the data
        environment.show_sensorData()  ## showing the data
    environment.map.blit(environment.infomap,(0,0)) ## drawing the data on info map
    pygame.display.update()

<Surface(1200x600x32 SW)>

0
15
58
99
137
172
202
234
258
284
313
337
363
385
413
436
465
489
521
558
586
610
632
652
677
698
717
734
749
766
779
797
819
845
870
897
920
939
963
984
1005
1029
1055
1079
1102
1127
1149
1176
1203
1224
1247
1271
1293
1322
1352
1382
1414
1447
1486
1493
1545
1597
1645
1699
1751
1802
1841
1847
1887
1922
1952
1981
2004
2026
2046
2065
2083
2102
2119
2131
2147
2167
2176
2186
2197
2208
2222
2269
2309
2351
2386
2413
2447
2470
2505
2535
2559
2564
2569
2615
2659
2698
2736
2774
2814
2846
2875
2904
2931
2961
2989
3014
3036
3053
3073
3098
3127
3159
3196
3235
3272
3308
3338
3370
3396
3416
3440
3469
3495
3514
3533
3549
3586
3625
3667
3712
3755
3796
3800
3825
3846
3864
3878
3893
3903
3912
3921
3935
3945
3964
3998
4002
4050
4070
4087
4102
4119
4137
4155
4178
4217
4270
4296
4311
4319
4325
4331
4335
4339
4344
4348
4351
4355
4359
4361
4363
4365


TypeError: 'bool' object is not iterable